## Overlays maps of p-values.

We assume the p-value maps have already been saved as tiff files.

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import matplotlib.pyplot as plt
import numpy as np

import tifffile

from janelia_core.visualization.volume_visualization import make_rgb_three_ch_z_plane_movie

Unable to import moviepy.  Minor functionality will not be available.


In [4]:
%matplotlib qt

In [5]:
# Paths to .tiff files to generate maps for
r_file = '/Volumes/bishoplab/projects/keller_vnc/results/whole_brain_stats/organized/state_dependence/B/B_state_dependence_ref_Q_cut_off_time_5_4997_mt_both_dff_1_5_5_long_bl_p_vls.tiff'
g_file = r'/Volumes/bishoplab/projects/keller_vnc/results/whole_brain_stats/organized/state_dependence/F/F_state_dependence_ref_Q_cut_off_time_5_4997_mt_both_dff_1_5_5_long_bl_p_vls.tiff'
b_file = r'/Volumes/bishoplab/projects/keller_vnc/results/whole_brain_stats/organized/state_dependence/TL/TL_state_dependence_ref_Q_cut_off_time_5_4997_mt_both_dff_1_5_5_long_bl_p_vls.tiff'

# Labels to associate with each color
r_label = 'Backwards'
g_label = 'Forwards'
b_label = 'Turn Left'

# Specify max p-value that receives a non-black color
max_p_vl = .05

# Specify lower percentile we use for choosing p-values that colors saturate at (values should be between 0 and 100)
min_p_val_percentile = 1.0

## Load the maps

In [6]:
maps = [tifffile.imread(f) if f is not None else None for f in [r_file, g_file, b_file] ]

## Convert the maps to log values

In [7]:
log_maps = [np.log10(m) if m is not None else None for m in maps]

## Merge the maps into an RGB color image

In [8]:
for m in log_maps:
    if m is not None:
        map_shape = m.shape
        
rgb_map_shape = list(map_shape) + [3]

In [9]:
labels = [r_label, g_label, b_label]

rgb_map = np.zeros(rgb_map_shape)
cmaps = [None]*3

for m_i, m in enumerate(log_maps):
    if m is not None:
        p_min = np.nanpercentile(m, min_p_val_percentile)
        p_max = np.log10(max_p_vl)
        p_range = p_max - p_min
    
        norm_m = 1 - (m - p_min)/p_range
        norm_m[norm_m > 1.0] = 1.0
        norm_m[norm_m < 0.0] = 0.0
        norm_m[np.isnan(norm_m)] = 0.0
    
        rgb_map[:,:,:, m_i] = norm_m
        
        cmaps[m_i] = {'label': labels[m_i], 'dark_vl': p_max, 'bright_vl': p_min}
    
    

## Generate movie

In [10]:
rgb_planes = [rgb_map[z_i, :,:,:] for z_i in range(rgb_map.shape[0])]

In [34]:
make_rgb_three_ch_z_plane_movie(z_imgs=rgb_planes, save_path = r'/Users/bishopw/Desktop/movies/test.mp4', 
                                cmaps=cmaps)

Unable to import moviepy.  Minor functionality will not be available.


RuntimeError: Requested MovieWriter (ffmpeg) not available

In [ ]:
plt.figure()
plt.imshow(rgb_map[12,:,:])

In [ ]:
plt.figure()
plt.imshow(log_maps[1][16, :,:])
plt.colorbar()

In [ ]:
plt.figure()
plt.imshow(rgb_map[16,:,:,1])

In [ ]:
cmaps

In [17]:
np.linspace(0, 1, 10)

Unable to import moviepy.  Minor functionality will not be available.


array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])